In [1]:
import sys
import pandas as pd
import numpy as np
import re
import time
import os
import csv
import tabulate
import matplotlib.pyplot as plt

from sklearn import metrics

from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn import svm, neighbors
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier #RandomizedLasso
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessRegressor 
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.semi_supervised import LabelPropagation
from sklearn.neural_network import MLPClassifier

In [2]:
PATH = '~/Documents/GitHub/march-madness/'
train = pd.read_csv('train_2010_2017.csv')

In [3]:
test = pd.read_csv('test_2018.csv')

In [4]:
## TAKE OUT ALL ADDED DATA 
takeout_vars = [u'PomeroyRank', u'Conf', u'AdjEM', u'AdjO',
       u'AdjD', u'AdjT', u'Luck', u'SOSAdjEM', u'OppO', u'OppD', u'NCSOSAdjEM',
       u'MooreRank', u'MooreSOS', u'MoorePR',u'OppPomeroyRank', u'OppConf', u'OppAdjEM', u'OppAdjO', u'OppAdjD',
       u'OppAdjT', u'OppLuck', u'OppSOSAdjEM', u'OppOppO', u'OppOppD',
       u'OppNCSOSAdjEM', u'OppMooreRank', u'OppW', u'OppL', u'OppT',
       u'OppMooreSOS', u'OppMoorePR',]
train = train.drop(takeout_vars, axis=1)
test = test.drop(takeout_vars, axis=1)

ValueError: labels [u'OppT'] not contained in axis

In [ ]:
# from fragile families open source code 
def factorize(df):
    """Convert features of type 'object', e.g. 'string', to categorical
    variables or factors."""
    for col in df.columns:
        if df.loc[:,col].dtype == object:
            factors, values = pd.factorize(df[col])
            df.loc[:,col] = factors
    return df

DROP THE WLT COLUMNS TO AVOID THE ISSUE RN 

In [ ]:
train = factorize(train)
test = factorize(test)
train = train.drop(['W', 'L', 'T'], axis=1)
test = test.drop(['W', 'L', 'T'], axis=1)

In [ ]:
y_train = train['Outcome']
X_train = train.drop('Outcome', axis=1)

In [ ]:
y_test = test['Outcome']
X_test = test.drop('Outcome', axis=1)

In [ ]:
def plot_confusion(cm, y_labels, cmap=plt.cm.Blues, filename='untitled.png'):
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.show()

In [ ]:
def bin_test(x_data, x_labels, y_data, y_labels):
    starttime = time.time()

    # binary models
    models = ["BNB", "GNB", "LDA","SVM_L", "SVM_G", "5NN", "LR2", "P2", "SGD","ADA", "DT", "RF", "DPGMM", "ET", "GMM", "MLP"] #"SVM_L", "SVM_G", "P2", "DT",  "ADA_R", 
    clfs = [BernoulliNB(), \
            GaussianNB(), \
            LinearDiscriminantAnalysis(), \
            svm.SVC(kernel = 'linear', probability=True), \
            svm.SVC(kernel='rbf', probability=True), \
            neighbors.KNeighborsClassifier(n_neighbors=5), \
            LogisticRegression(), \
            Perceptron(penalty='l2',tol=None,max_iter=1000), \
            SGDClassifier(tol=0.0001, power_t=0.4, average=True), \
            AdaBoostClassifier(base_estimator=None, n_estimators=100), \
            DecisionTreeClassifier(), \
            RandomForestClassifier(oob_score=True),  \
            BayesianGaussianMixture(n_components=2,max_iter=1000, weight_concentration_prior_type='dirichlet_process', tol=0.0001), \
            ExtraTreesClassifier(bootstrap=True, oob_score=True, n_estimators=4), \
            GaussianMixture(n_components=2, tol=0.0001, max_iter=1000, n_init=2), \
            MLPClassifier(activation='relu', alpha=0.00001, max_iter=1000)]


    results = []

    for i in range(len(clfs)):
        print "model being tested: {0}".format(models[i])
        time_start = time.time()
        clf = clfs[i].fit(x_data, x_labels)
        predict = clf.predict(y_data)
        runtime = time.time() - time_start
        p = metrics.precision_score(y_labels, predict)
        r = metrics.recall_score(y_labels, predict, average="macro")
        f = metrics.f1_score(y_labels, predict)

        # find outliers
        # data = [('challengeID', y_data['challengeID'].values),
        #         ('predicted', predict),
        #         ('label', y_labels.values)]
        # labels_and_predicted = pd.DataFrame.from_items(data)
        # outliers = y_data.merge(labels_and_predicted, on='challengeID')
        # outliers = outliers[outliers['label'] != outliers['predicted']]

        # num_mislabeled = outliers.shape[0]
        # a_new = -1
        # p_new = -1
        # r_new = -1
        # f_new = -1

        # if (outliers['label'].unique().size > 1):
        #     # train separate model on outliers
        #     mislabeled_labels = outliers['label']
        #     mislabeled_samples = outliers.drop(['label', 'predicted'], axis=1)

        #     (train_vars,validate_vars,train_outcomes,validate_outcomes) = train_test_split(mislabeled_samples,mislabeled_labels,test_size=0.2)

        #     clf_new = clfs[i].fit(train_vars, train_outcomes)
        #     validate_predicted = clf_new.predict(validate_vars)

        #     # evaluate
        #     a_new = metrics.accuracy_score(validate_outcomes, validate_predicted)
        #     p_new = metrics.precision_score(validate_outcomes, validate_predicted)
        #     r_new = metrics.recall_score(validate_outcomes, validate_predicted, average="macro")
        #     f_new = metrics.f1_score(validate_outcomes, validate_predicted)

        # results.append([models[i], a, p, r, f, runtime, num_mislabeled, a_new, p_new, r_new, f_new])
        results.append([models[i], p, r, f, runtime])
        # create confusion matrix 
        cm = metrics.confusion_matrix(y_labels, predict)
        plot_confusion(cm, y_labels, filename='{0}_confusion.png'.format(models[i]))
    print tabulate.tabulate(results, headers=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score','Runtime'])
    print "Binary test took {0} secs".format(time.time() - starttime)
    return pd.DataFrame(data=results)

In [ ]:
prediction_results = bin_test(X_train, y_train, X_test, y_test)

In [ ]:
prediction_results.to_csv('~/Documents/GitHub/march-madness/prediction_results')
prediction_results = np.round(prediction_results, decimals=2)
print prediction_results.to_latex(index=False, header=True)

# LOGISTIC REGRESSION

In [ ]:
# get only 2018 data first, for test set 
test2018 = test[test['Season']==2018]
# get only round one data 
test2018 = test2018[test2018['Round']==1]

test2018_y = test2018['Outcome']
test2018_X = test2018.drop('Outcome', axis=1)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
predict = clf.predict(test2018_X)
a = metrics.accuracy_score(test2018_y, predict)
p = metrics.precision_score(test2018_y, predict)
r = metrics.recall_score(test2018_y, predict, average="macro")
f = metrics.f1_score(test2018_y, predict)
result = [a, p, r, f]
print result
weights = clf.coef_
idx = np.argsort(weights)
weights = weights[0,idx]
features = np.asarray(X_test.columns)
features = features[idx]
weights = pd.DataFrame(data=weights)
features = pd.DataFrame(data=features)
FW = pd.concat([features, weights])

In [ ]:
print predict

In [ ]:
# normal LR 
clf = LogisticRegression()
clf.fit(X_train, y_train)
predict = clf.predict(X_test)
a = metrics.accuracy_score(y_test, predict)
p = metrics.precision_score(y_test, predict)
r = metrics.recall_score(y_test, predict, average="macro")
f = metrics.f1_score(y_test, predict)
result = [a, p, r, f]
print result

In [ ]:
FW

# EXTRA TREES

In [ ]:
clf = ExtraTreesClassifier(bootstrap=True, oob_score=True, n_estimators=4)
clf.fit(X_train, y_train)
predict = clf.predict(X_test)
a = metrics.accuracy_score(y_test, predict)
p = metrics.precision_score(y_test, predict)
r = metrics.recall_score(y_test, predict, average="macro")
f = metrics.f1_score(y_test, predict)
result = [a, p, r, f]
print result
weights = clf.feature_importances_
idx = np.argsort(weights)
weights = weights[idx]
features = np.asarray(X_test.columns)
features = features[idx]
weights = pd.DataFrame(data=weights)
features = pd.DataFrame(data=features)
FW = pd.concat([features, weights], axis=1)
FW.head

In [ ]:
predict

# SVM, linear

In [ ]:
clf = svm.SVC(kernel='linear',probability=True)
clf.fit(X_train, y_train)
predict = clf.predict(X_test)
a = metrics.accuracy_score(y_test, predict)
p = metrics.precision_score(y_test, predict)
r = metrics.recall_score(y_test, predict, average="macro")
f = metrics.f1_score(y_test, predict)
result = [a, p, r, f]
print result

In [ ]:
weights = clf.coef_
idx = np.argsort(weights)
weights = weights[0,idx]
features = np.asarray(X_test.columns)
features = features[idx]
weights = pd.DataFrame(data=weights)
features = pd.DataFrame(data=features)
FW = pd.concat([features, weights])
FW.head()

In [5]:
from sklearn.utils.testing import all_estimators

estimators = all_estimators()

for name, class_ in estimators:
    if hasattr(class_, 'predict_proba'):
        print(name)

AdaBoostClassifier
BaggingClassifier
BayesianGaussianMixture
BernoulliNB
CalibratedClassifierCV
DPGMM
DecisionTreeClassifier
ExtraTreeClassifier
ExtraTreesClassifier
GMM
GaussianMixture
GaussianNB
GaussianProcessClassifier
GradientBoostingClassifier
KNeighborsClassifier
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LogisticRegression
LogisticRegressionCV
MLPClassifier
MultinomialNB
NuSVC
QuadraticDiscriminantAnalysis
RandomForestClassifier
SGDClassifier
SVC
VBGMM
_BinaryGaussianProcessClassifierLaplace
_ConstantPredictor
_DPGMMBase
_GMMBase


/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [ ]:
clf = SGDClassifier(tol=0.0001, power_t=0.4, average=True)
clf.fit(X_train, y_train)
predict = clf.predict(X_test)
p = metrics.precision_score(y_test, predict)
r = metrics.recall_score(y_test, predict, average="macro")
f = metrics.f1_score(y_test, predict)
result = [p, r, f]
print result